<a href="https://colab.research.google.com/github/FilippMaksimov/Net.RecSystems/blob/main/Maksimov_F_Rec_Sys01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [18]:
# Обработка жанров

movies['genres'] = movies['genres'].fillna('')  # Заменяем NaN на пустую строку
# Применяем TF-IDF к жанрам
tfidf_genres = TfidfVectorizer(tokenizer=lambda x: x.split('|'))
genres_tfidf_matrix = tfidf_genres.fit_transform(movies['genres'])
# Преобразуем TF-IDF матрицу в DataFrame
genres_tfidf_df = pd.DataFrame(genres_tfidf_matrix.toarray(), columns=tfidf_genres.get_feature_names_out())

# Обработка тегов

tags['tag'] = tags['tag'].fillna('').astype(str)  # Заменяем NaN на пустую строку
tags_grouped = tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()
# Применяем TF-IDF к тегам
tfidf_tags = TfidfVectorizer()
tags_tfidf_matrix = tfidf_tags.fit_transform(tags_grouped['tag'])
# Преобразуем TF-IDF матрицу в DataFrame
tags_tfidf_df = pd.DataFrame(tags_tfidf_matrix.toarray(), columns=tfidf_tags.get_feature_names_out())
tags_tfidf_df['movieId'] = tags_grouped['movieId']

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [15]:
genres_tfidf_df

,(no genres listed),action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,film-noir,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western
0,0.0,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0.0,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,0.575034,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9738,0.0,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,0.638968,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9739,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9740,0.0,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [16]:
tags_tfidf_df

,06,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001,250,...,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey,movieId
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,183611
1568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,184471
1569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187593
1570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187595


In [22]:
#Агрегация статистик по пользователям
user_stats = ratings.groupby('userId')['rating'].agg(['mean', 'median', 'var', 'count']).reset_index()
user_stats.columns = ['userId', 'user_mean_rating', 'user_median_rating', 'user_rating_variance', 'user_rating_count']

#Агрегация статистик по фильмам
movie_stats = ratings.groupby('movieId')['rating'].agg(['mean', 'median', 'var', 'count']).reset_index()
movie_stats.columns = ['movieId', 'movie_mean_rating', 'movie_median_rating', 'movie_rating_variance', 'movie_rating_count']

# 1. Добавляем TF-IDF жанров к таблице movies
movies = movies.join(genres_tfidf_df)

#Объединяем данные: рейтинги, статистики, TF-IDF
ratings_with_features = ratings.merge(user_stats, on='userId', how='left')
ratings_with_features = ratings_with_features.merge(movie_stats, on='movieId', how='left')
ratings_with_features = ratings_with_features.merge(movies, on='movieId', how='left')
ratings_with_features = ratings_with_features.merge(tags_tfidf_df, on='movieId', how='left').fillna(0)

In [23]:
#Подготовка признаков и целевой переменной
exclude_columns = ['userId', 'movieId', 'rating', 'timestamp', 'title', 'genres']
X = ratings_with_features.drop(columns=exclude_columns)
y = ratings_with_features['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Обучение модели
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

#Оценка модели
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE на тестовой выборке: {rmse}")

RMSE на тестовой выборке: 0.8062353210590316
